### Library Import and Early Configuration

In [5]:
import os
import torch
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm.notebook import tqdm # Menggunakan versi notebook dari tqdm untuk visualisasi yang lebih baik
import matplotlib.pyplot as plt
import shutil

# --- KONFIGURASI ---
# Direktori root yang berisi semua folder eksperimen Anda
ROOT_DIR = "dataset_eval"
# Direktori output utama untuk semua hasil estetika
MAIN_OUTPUT_DIR = "Output/Aesthetic"
# Model CLIP yang digunakan untuk menilai estetika
MODEL_ID = "openai/clip-vit-large-patch14"
# Berapa persen di atas rata-rata sebuah gambar dianggap "baik"
GOOD_THRESHOLD_MULTIPLIER = 1.05  # 1.05 = 5% di atas rata-rata
# Jumlah gambar teratas yang akan ditampilkan di plot
TOP_N_IMAGES_TO_PLOT = 10

print("✅ Konfigurasi dan pustaka berhasil dimuat.")

✅ Konfigurasi dan pustaka berhasil dimuat.


### Function Definition Helper

In [6]:
def load_aesthetic_model(device):
    """Memuat model penilaian estetika yang sudah dilatih."""
    print(f"Memuat model estetika: {MODEL_ID}...")
    model = CLIPModel.from_pretrained(MODEL_ID).to(device)
    processor = CLIPProcessor.from_pretrained(MODEL_ID)
    print("Model estetika berhasil dimuat.")
    return model, processor

def calculate_aesthetic_score(image, model, processor, device):
    """Menghitung skor estetika untuk satu gambar."""
    inputs = processor(images=image, return_tensors="pt", text="aesthetic")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        score = outputs.logits_per_image.item()
    return score

# --- FUNGSI YANG DIPERBARUI ---
def create_ranking_plot_with_images(df_ranked, image_dir, plot_path, average_score, good_threshold, top_n=10):
    """Membuat plot peringkat skor dengan menampilkan gambar-gambar teratas DAN garis ambang batas."""
    top_df = df_ranked.head(top_n)
    
    fig, ax = plt.subplots(figsize=(15, 10))
    
    # Plot utama untuk skor
    ax.plot(df_ranked.index + 1, df_ranked['aesthetic_score'], marker='o', linestyle='-', label='Skor Estetika', zorder=5)
    
    # --- PERUBAHAN DIMULAI DI SINI ---
    # Tambahkan garis horizontal untuk skor rata-rata
    ax.axhline(y=average_score, color='red', linestyle=':', linewidth=2, label=f'Skor Rata-rata ({average_score:.2f})', zorder=10)
    # Tambahkan garis horizontal untuk ambang batas "baik"
    ax.axhline(y=good_threshold, color='green', linestyle='--', linewidth=2, label=f'Ambang Batas Baik ({good_threshold:.2f})', zorder=10)
    # --- AKHIR PERUBAHAN ---

    ax.set_title(f'Peringkat Skor Estetika - {os.path.basename(image_dir)}', fontsize=16)
    ax.set_xlabel('Peringkat', fontsize=12)
    ax.set_ylabel('Skor Estetika', fontsize=12)
    ax.grid(True)
    ax.legend()
    
    # Tambahkan gambar-gambar teratas ke dalam plot
    for i, (idx, row) in enumerate(top_df.iterrows()):
        try:
            img_path = os.path.join(image_dir, row['filename'])
            img = Image.open(img_path)
            
            img_ax = fig.add_axes([0.08 + i * 0.085, 0.85, 0.08, 0.08])
            img_ax.imshow(img)
            img_ax.set_title(f"#{i+1}", fontsize=10)
            img_ax.axis('off')
        except FileNotFoundError:
            print(f"Gambar tidak ditemukan untuk diplot: {row['filename']}")
            
    plt.tight_layout(rect=[0, 0, 1, 0.85])
    plt.savefig(plot_path)
    plt.close()

print("✅ Fungsi-fungsi bantuan telah didefinisikan (dengan pembaruan pada fungsi plot).")

✅ Fungsi-fungsi bantuan telah didefinisikan (dengan pembaruan pada fungsi plot).


### Model Load and Early Processing 

In [7]:
# Tentukan perangkat
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Menggunakan device: {device}")

# Muat model (hanya perlu dilakukan sekali)
model, processor = load_aesthetic_model(device)

# Cari semua folder eksperimen yang akan dievaluasi
experiment_folders = [d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))]

if not experiment_folders:
    print(f"Tidak ada folder eksperimen ditemukan di '{ROOT_DIR}'.")
else:
    print(f"Ditemukan {len(experiment_folders)} eksperimen untuk dievaluasi: {experiment_folders}")

Menggunakan device: cuda
Memuat model estetika: openai/clip-vit-large-patch14...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Model estetika berhasil dimuat.
Ditemukan 5 eksperimen untuk dievaluasi: ['exp1_sd15_2050', 'exp2_sd15_3050', 'exp3_sd15_3050_v2', 'exp4_sd15_3050_v3', 'exp5_sdxl_a6000']


### Process and Visualization

In [8]:
if not experiment_folders:
    print("Tidak ada folder untuk diproses. Silakan periksa kembali direktori Anda.")
else:
    for exp_name in experiment_folders:
        exp_input_dir = os.path.join(ROOT_DIR, exp_name)
        exp_output_dir = os.path.join(MAIN_OUTPUT_DIR, exp_name)
        good_images_dir = os.path.join(exp_output_dir, "good_images")
        
        os.makedirs(good_images_dir, exist_ok=True)
        
        results = []
        image_files = [f for f in os.listdir(exp_input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if not image_files:
            print(f"Tidak ada gambar ditemukan di '{exp_input_dir}'. Melanjutkan ke folder berikutnya.")
            continue

        print(f"\n▶️ Memproses {len(image_files)} gambar dari eksperimen: {exp_name}")
        for filename in tqdm(image_files, desc=f"Menilai {exp_name}"):
            try:
                image_path = os.path.join(exp_input_dir, filename)
                image = Image.open(image_path).convert("RGB")
                score = calculate_aesthetic_score(image, model, processor, device)
                results.append({"filename": filename, "aesthetic_score": score})
            except Exception as e:
                print(f"Tidak dapat memproses {filename}: {e}")

        if not results:
            print(f"Tidak ada gambar yang berhasil dinilai di {exp_name}. Melanjutkan ke folder berikutnya.")
            continue

        df = pd.DataFrame(results)
        df.sort_values(by="aesthetic_score", ascending=False, inplace=True)
        df.reset_index(drop=True, inplace=True)

        csv_path = os.path.join(exp_output_dir, f'aesthetic_scores_{exp_name}.csv')
        df.to_csv(csv_path, index=False)
        
        average_score = df['aesthetic_score'].mean()
        good_threshold = average_score * GOOD_THRESHOLD_MULTIPLIER
        good_images_df = df[df['aesthetic_score'] >= good_threshold]
        
        print(f"Skor rata-rata: {average_score:.4f} | Ambang batas 'Baik': {good_threshold:.4f}")
        print(f"Menyimpan {len(good_images_df)} gambar 'baik' ke: {good_images_dir}")
        
        for _, row in good_images_df.iterrows():
            src_path = os.path.join(exp_input_dir, row['filename'])
            dst_path = os.path.join(good_images_dir, row['filename'])
            shutil.copy(src_path, dst_path)

        plot_path = os.path.join(exp_output_dir, f'ranked_plot_with_images_{exp_name}.png')
        
        # --- PERUBAHAN PADA PANGGILAN FUNGSI ---
        create_ranking_plot_with_images(df, exp_input_dir, plot_path, average_score, good_threshold, top_n=TOP_N_IMAGES_TO_PLOT)
        
        print(f"Plot peringkat disimpan di: {plot_path}")
        print(f"✅ Selesai memproses {exp_name}.")

    print("\n🎉 Semua eksperimen telah selesai dievaluasi!")


▶️ Memproses 60 gambar dari eksperimen: exp1_sd15_2050


Menilai exp1_sd15_2050:   0%|          | 0/60 [00:00<?, ?it/s]

Skor rata-rata: 14.4622 | Ambang batas 'Baik': 15.1853
Menyimpan 21 gambar 'baik' ke: Output/Aesthetic\exp1_sd15_2050\good_images


C:\Users\Pongo\AppData\Local\Temp\ipykernel_29460\2153476775.py:55: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 1, 0.85])


Plot peringkat disimpan di: Output/Aesthetic\exp1_sd15_2050\ranked_plot_with_images_exp1_sd15_2050.png
✅ Selesai memproses exp1_sd15_2050.

▶️ Memproses 60 gambar dari eksperimen: exp2_sd15_3050


...

Skor rata-rata: 12.5682 | Ambang batas 'Baik': 13.1966
Menyimpan 15 gambar 'baik' ke: Output/Aesthetic\exp3_sd15_3050_v2\good_images
Plot peringkat disimpan di: Output/Aesthetic\exp3_sd15_3050_v2\ranked_plot_with_images_exp3_sd15_3050_v2.png
✅ Selesai memproses exp3_sd15_3050_v2.

▶️ Memproses 60 gambar dari eksperimen: exp4_sd15_3050_v3


Menilai exp4_sd15_3050_v3:   0%|          | 0/60 [00:00<?, ?it/s]

Skor rata-rata: 13.5991 | Ambang batas 'Baik': 14.2790
Menyimpan 26 gambar 'baik' ke: Output/Aesthetic\exp4_sd15_3050_v3\good_images
Plot peringkat disimpan di: Output/Aesthetic\exp4_sd15_3050_v3\ranked_plot_with_images_exp4_sd15_3050_v3.png
✅ Selesai memproses exp4_sd15_3050_v3.

▶️ Memproses 60 gambar dari eksperimen: exp5_sdxl_a6000


Menilai exp5_sdxl_a6000:   0%|          | 0/60 [00:00<?, ?it/s]

Skor rata-rata: 12.4829 | Ambang batas 'Baik': 13.1070
Menyimpan 19 gambar 'baik' ke: Output/Aesthetic\exp5_sdxl_a6000\good_images
Plot peringkat disimpan di: Output/Aesthetic\exp5_sdxl_a6000\ranked_plot_with_images_exp5_sdxl_a6000.png
✅ Selesai memproses exp5_sdxl_a6000.

🎉 Semua eksperimen telah selesai dievaluasi!
